<a href="https://colab.research.google.com/github/littledumbo/bank-churn-prediction-model/blob/master/FINAL_Script_email_%2B_threshold_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install schedule

In [ ]:
import pandas as pd
import numpy as np
import pickle
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import logging
import requests
import schedule
import time

In [ ]:
logging.basicConfig(filename='churn_email.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
url = "https://raw.githubusercontent.com/RicardoMuchacho/bank-churn-prediction-model/master/xgb_churn_model.pkl"

response = requests.get(url)
if response.status_code == 200:
    with open("xgb_churn_model.pkl", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully!")
else:
    print("Failed to download file")

with open("xgb_churn_model.pkl", "rb") as f:
    model = pickle.load(f)

File downloaded successfully!


In [ ]:
url_data = "https://raw.githubusercontent.com/RicardoMuchacho/bank-churn-prediction-model/master/Customer_Data_Corrected.csv"


customer_data = pd.read_csv(url_data)

print(customer_data.columns)
customer_data.head()

Index(['customer_id', 'customer_name', 'email_address', 'CreditScore',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Satisfaction Score',
       'Card Type', 'Earned Point', 'Geography_France', 'Geography_Germany',
       'Geography_Spain'],
      dtype='object')


,customer_id,customer_name,email_address,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Satisfaction Score,Card Type,Earned Point,Geography_France,Geography_Germany,Geography_Spain
0,CUST1000,Ricardo Muchacho,ricardojrmuchacho2@gmail.com,452,0,43,10,21098.85,4,1,0,193429.68,0,4,3,289,False,True,True
1,CUST1001,Antoine Maisonhaute,antoinemaisonhaute59@gmail.com,785,1,61,4,21098.85,4,1,1,193429.68,0,3,3,224,True,True,True
2,CUST1002,Elihu Cardenas,elihu.ca@gmail.com,698,0,51,9,21098.85,4,0,0,193429.68,1,1,2,249,True,False,False
3,CUST1003,Sergio Martinez,sergio19959@gmail.com,620,1,27,6,21098.85,4,1,1,193429.68,0,1,2,413,False,True,False
4,CUST1004,Ricardo Muchacho,ricardojrmuchacho2@gmail.com,456,1,79,9,21098.85,4,0,1,193429.68,0,4,2,669,False,False,True


In [ ]:
df = customer_data.drop(columns=['customer_id', 'customer_name', 'email_address','Exited'])

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CreditScore         50 non-null     int64  
 1   Gender              50 non-null     int64  
 2   Age                 50 non-null     int64  
 3   Tenure              50 non-null     int64  
 4   Balance             50 non-null     float64
 5   NumOfProducts       50 non-null     int64  
 6   HasCrCard           50 non-null     int64  
 7   IsActiveMember      50 non-null     int64  
 8   EstimatedSalary     50 non-null     float64
 9   Satisfaction Score  50 non-null     int64  
 10  Card Type           50 non-null     int64  
 11  Earned Point        50 non-null     int64  
 12  Geography_France    50 non-null     bool   
 13  Geography_Germany   50 non-null     bool   
 14  Geography_Spain     50 non-null     bool   
dtypes: bool(3), float64(2), int64(10)
memory usage: 5.0 KB


In [ ]:
churn_probabilities = model.predict_proba(df)[:, 1]

In [ ]:
threshold1 = 0.5
threshold2 = 0.65
threshold3 = 0.8

low_risk_indices = np.where(churn_probabilities > threshold1)[0]
medium_risk_indices = np.where(churn_probabilities > threshold2)[0]
high_risk_indices = np.where(churn_probabilities > threshold3)[0]

low_risk_customers = customer_data.iloc[low_risk_indices]
medium_risk_customers = customer_data.iloc[medium_risk_indices]
high_risk_customers = customer_data.iloc[high_risk_indices]

In [ ]:
EMAIL_ADDRESS = 'antoinemaisonhaute59@gmail.com'
EMAIL_PASSWORD = 'vwml iykz lycp ozdn'
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587

In [ ]:
def send_email_low(to_address, customer_name):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_address
        msg['Subject'] = 'Un correo para agradecerte por estar con nosotros'

        body = f"""
        Querido {customer_name},

        Sabemos que has estado con nosotros durante algún tiempo y queremos recompensar tu fidelidad y mostrarte nuestro agradecimiento.
        Ahora tienes disponibles algunas ofertas especiales para ti y tus seres queridos en préstamos familiares, de automóvil o vivienda.
        ¡No te los pierdas!

        Esperamos saber de ti pronto y poder hablar sobre nuestro futuro y éxitos en común.

        No dudes en ponerte en contacto con nuestros servicios, ¡te guiaremos a través de todos tus beneficios!

        Saludos cordiales,
        Tu servicio de atención al cliente
        """
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(f"Email mandado a {to_address}")
    except Exception as e:
        logging.error(f"Failó mandar el email a {to_address}: {e}")

                    email_address        customer_name
0    ricardojrmuchacho2@gmail.com     Ricardo Muchacho
1  antoinemaisonhaute59@gmail.com  Antoine Maisonhaute
2              elihu.ca@gmail.com       Elihu Cardenas
3           sergio19959@gmail.com      Sergio Martinez
4    ricardojrmuchacho2@gmail.com     Ricardo Muchacho


In [ ]:
for index, row in low_risk_customers.iterrows():
    send_email_low(row['email_address'], row['customer_name'])

In [ ]:
def send_email_medium(to_address, customer_name):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_address
        msg['Subject'] = 'Un correo para agradecerte por estar con nosotros'

        body = f"""
        Querido {customer_name},

        Sabemos que has estado con nosotros durante algún tiempo y queremos recompensar tu fidelidad y mostrarte nuestro agradecimiento.
        Ahora tienes disponibles algunas ofertas especiales para ti y tus seres queridos en planes de pensiones y seguros de salud.
        ¡No te los pierdas!

        Esperamos saber de ti pronto y poder hablar sobre nuestro futuro y éxitos en común.

        No dudes en ponerte en contacto con nuestros servicios, ¡te guiaremos a través de todos tus beneficios!

        Saludos cordiales,
        Tu servicio de atención al cliente
        """
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(f"Email mandado a {to_address}")
    except Exception as e:
        logging.error(f"Failó mandar el email a {to_address}: {e}")

In [ ]:
for index, row in medium_risk_customers.iterrows():
    send_email_medium(row['email_address'], row['customer_name'])

In [ ]:
def send_email_high(to_address, customer_name):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_address
        msg['Subject'] = 'Un correo para agradecerte por estar con nosotros'

        body = f"""
        Querido {customer_name},

        Sabemos que has estado con nosotros durante algún tiempo y queremos recompensar tu fidelidad y mostrarte nuestro agradecimiento.
        Ahora tienes disponibles algunas ofertas especiales para ti y tus seres queridos en préstamos familiares, de automóvil o vivienda,
        así como descuentos muy interesantes en planes de pensiones y seguros de salud. ¡No te los pierdas!

        Esperamos saber de ti pronto y poder hablar sobre nuestro futuro y éxitos en común.

        No dudes en ponerte en contacto con nuestros servicios, ¡te guiaremos a través de todos tus beneficios!

        Saludos cordiales,
        Tu servicio de atención al cliente
        """
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)

        logging.info(f"Email mandado a {to_address}")
    except Exception as e:
        logging.error(f"Failó mandar el email a {to_address}: {e}")

In [ ]:
for index, row in high_risk_customers.iterrows():
    send_email_high(row['email_address'], row['customer_name'])